In [61]:
#%pip install agentpy owlready2


In [62]:
import agentpy as ap
from matplotlib import pyplot as plt
import IPython
import random
from owlready2 import *

In [63]:
onto = get_ontology("file://onto.owl")


In [64]:
#ONTOLOGIA
#onto.delete()
with onto:

    class Entity(Thing):
        pass

    class Vaccum(Entity):
        pass

    class Dirt(Entity):
        pass

    class Box(Entity):
        boxStack=1
        pass

    class Place(Thing):
        pass

    class Position(Thing):
        pass

    class has_place(ObjectProperty, FunctionalProperty):
        domain = [Entity]
        range = [Place]

    class has_position(DataProperty, FunctionalProperty):
        domain = [Place]
        range = [str]

        pass



In [65]:
class BoxPile(ap.Agent):

    def setup(self):
        self.boxStack = 1
        self.agentType = 4
        self.first_step = True
        self.pos = None
        self.stacked=False
        pass
    
    def add_box(self):
        self.boxStack += 1
        pass

    def step(self):
        if self.first_step:
          self.pos = self.model.grid.positions[self]
          self.first_step = False
        pass

    def update(self):
        pass

    def end(self):
        pass

In [66]:
class Robot(ap.Agent):
    def see(self, e):
        self.per = []
        vecinos = e.neighbors(self, 1)
        print(f"Agent position: {e.positions[self]}")
        for vecino in vecinos:
            box = next((b for b in self.model.boxes if b.pos == e.positions[vecino]), None)
            if box:
                self.per.append(Box(has_place=Place(has_position=str(e.positions[vecino])), boxStack=box.boxStack))
            else:
                self.per.append(Box(has_place=Place(has_position=str(e.positions[vecino])), boxStack=0))
        
    def next(self):
        action_taken = False
    # Por cada acción
        for act in self.actions:
        # Por cada regla
            for rule in self.rules:
                # Si la acción es válida dada la regla
                if rule(act):
                # Ejecuta la acción
                    act()
                    action_taken = True
                    break
            if action_taken:
                break
    
    # If no action was taken, move randomly
        if not action_taken:
            self.move_random()
        
        #REGLAS PARA GRAB
    def rule_grabBox(self,act):
        #REGLA PARA TOMAR CAJA
        
         # Validador de regla
        validador = [False, False, False]

        # Proposición 1: Si hay box en la posición actual
        for boxes in self.model.boxes:
            if boxes.pos == self.model.grid.positions[self] and boxes.boxStack == 1 and boxes.stacked==False:
                validador[0] = True
        
        if self.carryingBox==False :
            validador[1] = True
        
        # Proposición 2: Si la acción es la de grabBox

        if act == self.grabBox:
            validador[2] = True

        return sum(validador) == 3
    
    
    def rule_mov_N_grab(self,act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][0] - 1 and (boxes.boxStack == 1 or (not self.carryingBox and boxes.boxStack == 0)):
                validador[0] = True
        
        if self.carryingBox==False:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el norte
        if act == self.move_N:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_S_grab(self,act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][0] + 1 and (boxes.boxStack == 1 or (not self.carryingBox and boxes.boxStack == 0)):
                validador[0] = True
        
        if self.carryingBox==False:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el Sur
        if act == self.move_S:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_E_grab(self,act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][1] + 1 and (boxes.boxStack == 1 or (not self.carryingBox and boxes.boxStack == 0)):
                validador[0] = True
        
        if self.carryingBox==False:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el norte
        if act == self.move_E:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_W_grab(self,act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][1] - 1 and (boxes.boxStack == 1 or (not self.carryingBox and boxes.boxStack == 0)):
                validador[0] = True
        
        if self.carryingBox==False:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el Sur
        if act == self.move_W:
            validador[2] = True

        return sum(validador) == 3  
    
    
    
    def rule_mov_buscar_grabBox(self,act):

        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si no hay box en el entorno
        
        if len(self.per) <= 0:
            validador[0] = True
            
        if self.carryingBox==False:
            validador[1] = True
            
        

        # Proposición 2: Si la acción es la de moverse aleatoriamente
        if act == self.move_random:
            validador[2] = True

        return sum(validador) == 3

    
    #REGLAS PARA STACK
    
    def rule_stackBox(self, act):
        # Validador de regla
        validador = [False, False, False]

        # Proposición 1: Si hay una caja en la posición actual para apilar
        for boxes in self.model.boxes:
            if boxes.pos == self.model.grid.positions[self] and 1 <= boxes.boxStack <= 2:
                validador[0] = True

        # Proposición 2: Si el robot tiene una caja
        if self.carryingBox:
            validador[1] = True

        # Proposición 3: Si la acción es la de stackBox
        if act == self.stackBox:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_N_stackBox(self, act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][0]-1 and 1<= boxes.boxStack < 5:
                validador[0] = True
        
        if self.carryingBox==True:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el norte
        if act == self.move_N:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_S_stackBox(self, act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[0] == self.model.grid.positions[self][0]+1 and 1<= boxes.boxStack < 5:
                validador[0] = True
        
        if self.carryingBox==True:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el sur
        if act == self.move_S:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_E_stackBox(self, act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[1] == self.model.grid.positions[self][1]+1 and 1<= boxes.boxStack < 5:
                validador[0] = True
        
        if self.carryingBox==True:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el este
        if act == self.move_E:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_mov_W_stackBox(self, act):
        # Validador de regla
        validador = [False,False,False]

        # Proposición 1: Si hay box en la posición Norte
        for boxes in self.per:
            if eval(boxes.has_place.has_position)[1] == self.model.grid.positions[self][1]-1 and 1<= boxes.boxStack < 5:
                validador[0] = True
        
        if self.carryingBox==True:
            validador[1] = True
        
        
        # Proposición 2: Si la acción es la de moverse hacia el sur
        if act == self.move_W:
            validador[2] = True

        return sum(validador) == 3
    
    def rule_buscar_stackBox(self, act):
        validador = [False, False]

        if self.carryingBox == True and all(box.boxStack != 1 for box in self.per):
            validador[0] = True

        if act == self.move_random:
            validador[1] = True

        return sum(validador) == 2

    #SIMULACIÓN DE AGENTE
    
    
    
    def setup(self):
        """
        Función de inicialización
        """

        self.agentType = 0 # Tipo de agente
        self.carryingBox = False
        self.direction = (-1,0) # Dirección inicial
        self.previous_position = None
        self.last_positions = [] 
        # Acciones del agente
        self.actions = (
            self.move_N,
            self.move_S,
            self.move_E,
            self.move_W,
            self.move_random,
            self.grabBox,
            self.stackBox,
            
        )
        # Reglas del agente
        self.rules = (
            self.rule_grabBox,
            self.rule_mov_N_grab,
            self.rule_mov_S_grab,
            self.rule_mov_E_grab,
            self.rule_mov_W_grab,
            self.rule_mov_buscar_grabBox,
            self.rule_mov_N_stackBox,
            self.rule_mov_S_stackBox,
            self.rule_mov_E_stackBox,
            self.rule_mov_W_stackBox,
            self.rule_stackBox,
            self.rule_buscar_stackBox
            
        )
        pass
    
    def check_if_stuck(self):
        if len(set(self.last_positions)) == 1 and len(self.last_positions) >= 5:
            self.move_random()
            self.last_positions.clear()

    
    def step(self):
        self.see(self.model.grid)
        self.next()
        
        current_pos = self.model.grid.positions[self]
        self.last_positions.append(current_pos)
        if len(self.last_positions) > 5:
            self.last_positions.pop(0)
        
        self.check_if_stuck()
    
    def update(self):
        pass

    def end(self):
        pass
    
    
        #ACCIONES
    def move_N(self):
        """
        Función de movimiento hacia el norte
        """
        self.direction = (-1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_S(self):
        """
        Función de movimiento hacia el sur
        """
        self.direction = (1,0) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_E(self):
        """
        Función de movimiento hacia el este
        """
        self.direction = (0,1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_W(self):
        """
        Función de movimiento hacia el oeste
        """
        self.direction = (0,-1) #Cambio de dirección
        self.forward() # Caminar un paso hacia adelante

    def move_random(self):
        """
        Función de movimiento aleatorio
        """
        # Rotaciones aleatorias
        for _ in range(random.randint(0,4)):
            self.turn()
        self.forward() # Caminar un paso hacia adelante
    
    
    def forward(self):
        """
        Función de movimiento
        """
        self.model.grid.move_by(self, self.direction)
        pass

    def turn(self):
        """
        Función de rotación
        """
        if self.direction == (-1,0):
            self.direction = (0,1) #Hacia Este
        elif self.direction == (0,1):
            self.direction = (1,0)  #Hacia Sur
        elif self.direction == (1,0):
            self.direction = (0,-1) #Hacia Oeste
        elif self.direction == (0,-1):
            self.direction = (-1,0) #Hacia Norte
        pass
    
    
    def grabBox(self):
        if not self.carryingBox:
            for box in self.model.boxes[:]:  # Create a copy of the list to iterate over
                if (
                    box.pos == self.model.grid.positions[self]
                    and box.boxStack == 1
                    and not self.carryingBox
                ):
                    print(f"Encountered a box at {box.pos}. Carrying box = {self.carryingBox}")
                    self.model.grid.remove_agents(box)
                    self.model.boxes.remove(box)  # Remove the box from the model's list of boxes
                    self.carryingBox = True
                    print("Grabbed a box")
                    return
                    
    
    
    def stackBox(self):
        if self.carryingBox:
            current_position = self.model.grid.positions[self]
            
            # Check if there's already a box pile at the current position
            existing_box = next((box for box in self.model.boxes if box.pos == current_position), None)
            
            if existing_box:
                if 1 <= existing_box.boxStack < 5:
                    existing_box.add_box()
                    self.carryingBox = False
                    print(f"Stacked the box. Pile now has {existing_box.boxStack} boxes.")
                    return True
                else:
                    print(f"Cannot stack the box here, the pile has {existing_box.boxStack} boxes")
            else:
                # Create a new box at the current position
                new_box = Box(pos=current_position, model=self.model)
                new_box.boxStack = 2  # Start with 2 because we're adding our carried box
                self.model.grid.add_agents(new_box)
                self.model.boxes.append(new_box)
                self.carryingBox = False
                print(f"Created a new stack with 2 boxes at {current_position}")
                return True
        return False
                        
             

    
  

In [67]:
class RobotModel(ap.Model):
    def setup(self):
        
        self.robots = ap.AgentList(self,self.p.robots,Robot)
        self.boxes = ap.AgentList(self, self.p.boxes, BoxPile)
        
        self.grid = ap.Grid(self, (self.p.M, self.p.N), track_empty=True)

        self.grid.add_agents(self.robots, random=True, empty=True)
        self.grid.add_agents(self.boxes, random=True, empty=True)
        
        pass
    
    def step(self):
        self.boxes.step()
        self.robots.step()
        
        pass
    
    def update(self):
        pass
    def end(self):
        pass

                                  

In [68]:
parameters = {
    'M': 20,
    'N': 20,
    "steps": 100,
    "robots": 1,
    'boxes': 20
    
}

In [69]:
def animation_plot(model, ax):
    """
    Función de animación
    @param model: modelo
    @param ax: axes (matplotlib)
    """
    # Definición de atributo para tipo de agente
    agent_type_grid = model.grid.attr_grid('agentType')
    # Definición de gráfico con colores (de acuerdo al tipo de agente)
    ap.gridplot(agent_type_grid, cmap='Accent', ax=ax)
    # Definición de título del gráfico
    ax.set_title(f"Robot Model \n Time-step: {model.t}, "
                 f"Boxes: {len(model.boxes)}, Robots: {len(model.robots)}")
    
    

In [70]:
#SIMULATION:

#Create figure (from matplotlib)
fig, ax = plt.subplots()

#Create model
model = RobotModel(parameters)

#Run with animation
#If you want to run it without animation then use instead:
#model.run()
animation = ap.animate(model, fig, ax, animation_plot)
#This step may take a while before you can see anything

#Print the final animation
IPython.display.HTML(animation.to_jshtml())



Agent position: (14, 3)
Agent position: (14, 2)
Encountered a box at (14, 2). Carrying box = False
Grabbed a box
Agent position: (14, 2)
Agent position: (14, 1)
Agent position: (15, 1)
Agent position: (15, 2)
Agent position: (15, 1)
Agent position: (14, 1)
Agent position: (14, 0)
Agent position: (14, 0)
Agent position: (13, 0)
Agent position: (12, 0)
Agent position: (12, 1)
Stacked the box. Pile now has 2 boxes.
Agent position: (12, 1)
Agent position: (11, 1)
Agent position: (11, 0)
Agent position: (11, 1)
Agent position: (11, 2)
Agent position: (11, 1)
Agent position: (11, 0)
Agent position: (11, 1)
Agent position: (11, 2)
Agent position: (11, 3)
Agent position: (12, 3)
Agent position: (12, 4)
Agent position: (12, 3)
Agent position: (12, 2)
Agent position: (13, 2)
Agent position: (13, 1)
Agent position: (13, 0)
Agent position: (14, 0)
Agent position: (15, 0)
Agent position: (16, 0)
Agent position: (17, 0)
Agent position: (17, 0)
Agent position: (17, 0)
Agent position: (17, 0)
Agent po